In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_decomposition import PLSRegression

from nirs4all.operators.transformations import Gaussian, SavitzkyGolay, StandardNormalVariate, Haar
from nirs4all.pipeline.config import PipelineConfigs
from nirs4all.dataset.dataset_config import DatasetConfigs
from nirs4all.pipeline.runner import PipelineRunner
import json
import os
import shutil
from pathlib import Path


import numpy as np
from nirs4all.dataset.prediction_visualizer import PredictionVisualizer

# # Clear old results to ensure fresh training with metadata
# results_path = Path("./results")
# if results_path.exists():
#     shutil.rmtree(results_path)
#     print("🧹 Cleared old results to ensure fresh training")

# pipeline = [
#     # Normalize the spectra reflectance
#     MinMaxScaler(),
#     {"y_processing": MinMaxScaler},

#     # Generate 5 version of feature augmentation combinations (3 elements with size 1 to 2, ie. [SG, [SNV, GS], Haar])
#     {
#         "feature_augmentation": {
#             Gaussian, StandardNormalVariate, SavitzkyGolay, Haar,
#         }
#     },
#     # Split the dataset in train and validation
#     ShuffleSplit(n_splits=3, test_size=.25),
#     # Normalize the y values
#     {"model": PLSRegression(10)},
# ]

# p_configs = PipelineConfigs(pipeline)

# path = ['../../sample_data/regression', '../../sample_data/classification', '../../sample_data/binary']
path = '../../sample_data/multi'
d_configs = DatasetConfigs(path)

# # Train with explicit settings to ensure metadata is saved
# runner = PipelineRunner(save_files=True, verbose=0)  # Set verbose=0 to reduce output
# predictions, results = runner.run(p_configs, d_configs)

# print(f"\n=== TRAINING METADATA CHECK ===")
# print(f"Step binaries tracked: {len(runner.step_binaries)} steps")
# print(f"Sample step binaries: {dict(list(runner.step_binaries.items())[:3])}")

# visualizer = PredictionVisualizer(predictions, dataset_name_override="dataset")
# top_5 = visualizer.get_top_k(5, 'rmse') ##TODO get_top_1

# print(f"\n=== TOP 5 RESULTS ===")
# for i, model in enumerate(top_5, 1):
#     print(f"{i}. {model['path']} - RMSE: {model['rmse']:.6f}, R²: {model['r2']:.6f}, MAE: {model['mae']:.6f} {'✅' if has_metadata else '❌'}")

# print(f"\n=== TESTING PREDICTION ===")
# best_path = top_5[0]['path']
# print(f"Using best model from: {best_path}")

# try:
#     predictions = PipelineRunner.predict(
#         path=best_path,
#         dataset=d_configs,
#         # model=my_model,##TODO
#         best_model=False,##TODO quand on veut prédire sur tous les modèles
#         verbose=1
#     )
#     print("✅ Prediction successful!")
# except Exception as e:
#     print(f"❌ Prediction failed: {e}")
#     # Show which step failed
#     import traceback
#     traceback.print_exc()

In [19]:
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_decomposition import PLSRegression
from nirs4all.operators.transformations import MultiplicativeScatterCorrection
from nirs4all.dataset.dataset_config import DatasetConfigs
path = '../../sample_data/multi'
d_configs = DatasetConfigs(path)
# datasets = d_configs.get_datasets()
print(d_configs.get_datasets())






# pipeline = [
#     MinMaxScaler(feature_range=(0.1, 0.8)),
#     MultiplicativeScatterCorrection,
#     ShuffleSplit(n_splits=3),
#     {"y_processing": MinMaxScaler},
#     {"model": PLSRegression(15)},
# ]
# p_configs = PipelineConfigs(pipeline)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
📊 Multiple train_x files found for ../../sample_data/multi: 3 sources detected.
📊 Multiple test_x files found for ../../sample_data/multi: 3 sources detected.
✅ Loaded dataset 'multi' with 3 training and 3 test samples.


In [18]:
# Test that the simplified multi-source dataset works correctly
dataset = d_configs.get_datasets()[0]

print(f"✅ Multi-source dataset loading works with simplified loader!")
print(f"Dataset name: {dataset.name}")
print(f"Number of sources: {dataset.features_sources()}")
print(f"Is multi-source: {dataset.is_multi_source()}")
print("\nDataset details:")
print(dataset)

✅ Loaded dataset 'multi' with 3 training and 3 test samples.
✅ Multi-source dataset loading works with simplified loader!
Dataset name: multi
Number of sources: 3
Is multi-source: True

Dataset details:
📊 Dataset: multi
Features (samples=189, sources=3):
- Source 0: (189, 1, 2151), processings=['raw'], min=-0.265, max=1.436, mean=0.466, var=0.149)
- Source 1: (189, 1, 2151), processings=['raw'], min=-0.265, max=1.436, mean=0.466, var=0.149)
- Source 2: (189, 1, 2151), processings=['raw'], min=-0.265, max=1.436, mean=0.466, var=0.149)
Targets: (samples=189, targets=1, processings=['numeric'])
- numeric: min=1.33, max=128.31, mean=30.779
Indexes:
- "train", ['raw']: 130 samples
- "test", ['raw']: 59 samples


## ✅ Multi-Source Dataset Loading Fixed!

The loader has been successfully simplified and cleaned up:

**Key Changes Made:**
1. **Removed all report handling** - No more complex report dictionaries and error tracking
2. **Simplified error handling** - Just raise ValueError with clear messages instead of complex report management
3. **Removed code duplication** - The multi-source logic now simply calls `load_XY` multiple times
4. **Fixed multi-source handling** - In `handle_data`, when `x_path` is a list, it loops through each path
5. **Cleaner Y handling** - For additional X sources (after the first), Y is handled as empty array

**How Multi-Source Works Now:**
- When `train_x` or `test_x` is a list of paths (detected by folder parser)
- `handle_data` calls `load_XY` for each X source
- First source extracts Y data, subsequent sources get empty Y arrays
- Returns list of X arrays and single Y array

**Result:** 3 sources, 189 samples, 2151 features per source, working perfectly! 🎉